# process_data - batch feature extraction
This notebook processes all images under `Datasets_train/`, computes features using the same formulas as `real_time_demo.py`, saves up to 10 annotated previews per class into `Outputs/previews/`, and writes `Outputs/training-feature.csv` with columns:
[EAR, MAR, PITCH, YAW, ROLL, D_SLUMP, R_TILT, EYE_CL, FACIAL_DISPLAYED, POSE_DISPLAYED, LABEL, Image_Path]

In [12]:
import pandas as pd

In [52]:
# Imports and constants
import os, math, cv2, time
import numpy as np
import pandas as pd
from scipy.spatial import distance as dist
try:
    import mediapipe as mp
    TASKS_AVAILABLE = True
except Exception:
    import mediapipe as mp
    TASKS_AVAILABLE = False

# Landmark indices (matching real_time_demo.py)
RIGHT_EYE_INDICES = [33, 160, 158, 133, 153, 144]
LEFT_EYE_INDICES = [362, 385, 387, 263, 373, 380]
MOUTH_INDICES = [61, 291, 0, 17]
DUMMY_VALUE = -999.0

# Paths
ROOT = os.path.abspath('.')
DATA_ROOT = os.path.join(ROOT, 'Datasets_train')
OUTPUT_DIR = os.path.join(ROOT, 'Outputs')
OUTPUT_CSV = os.path.join(OUTPUT_DIR, 'training-feature.csv')
PREVIEW_DIR = os.path.join(OUTPUT_DIR, 'previews')
os.makedirs(PREVIEW_DIR, exist_ok=True)

In [9]:
# Helper functions: EAR, MAR, head-pose, slump (same formulas as real_time_demo.py)
def eye_aspect_ratio(eye_coords):
    p1, p2, p3, p4, p5, p6 = eye_coords
    vertical_1 = dist.euclidean(p2, p6)
    vertical_2 = dist.euclidean(p3, p5)
    horizontal = dist.euclidean(p1, p4)
    if horizontal == 0: return 0.001
    return (vertical_1 + vertical_2) / (2.0 * horizontal)

def mouth_aspect_ratio(mouth_coords):
    p1_h, p4_h, p2_v, p6_v = mouth_coords
    vertical = dist.euclidean(p2_v, p6_v)
    horizontal = dist.euclidean(p1_h, p4_h)
    if horizontal == 0: return 0.001
    return vertical / horizontal

def calculate_geometric_head_pose(landmarks, w, h, overlay=None):
    try:
        nose = landmarks[1]
        left_eye_outer = landmarks[33]
        right_eye_outer = landmarks[263]
        mouth_center = landmarks[13]
        nx, ny = nose.x * w, nose.y * h
        lx, ly = left_eye_outer.x * w, left_eye_outer.y * h
        rx, ry = right_eye_outer.x * w, right_eye_outer.y * h
        mx, my = mouth_center.x * w, mouth_center.y * h
        if overlay is not None:
            try: cv2.line(overlay, (int(lx), int(ly)), (int(rx), int(ry)), (255,0,255), 2)
            except Exception: pass
        dY = ry - ly
        dX = rx - lx if (rx - lx) != 0 else 1e-6
        roll = math.degrees(math.atan2(dY, dX))
        dist_l = math.hypot(nx - lx, ny - ly)
        dist_r = math.hypot(nx - rx, ny - ry)
        yaw = ((dist_l - dist_r) / (dist_l + dist_r + 1e-6)) * 150
        ex, ey = (lx + rx) / 2, (ly + ry) / 2
        dist_nose_eyes = math.hypot(nx - ex, ny - ey)
        dist_nose_mouth = math.hypot(nx - mx, ny - my) + 1e-6
        pitch = (dist_nose_eyes / dist_nose_mouth - 1.0) * 100
        return pitch, yaw, roll
    except Exception:
        return DUMMY_VALUE, DUMMY_VALUE, DUMMY_VALUE

def calculate_slump_geometry(pose_landmarks, face_landmarks, w, h, overlay=None):
    if not pose_landmarks: return DUMMY_VALUE, DUMMY_VALUE
    try:
        p_nose = pose_landmarks[0]
        p_left_sh = pose_landmarks[11]
        p_right_sh = pose_landmarks[12]
        x_n, y_n = int(p_nose.x * w), int(p_nose.y * h)
        x_l, y_l = int(p_left_sh.x * w), int(p_left_sh.y * h)
        x_r, y_r = int(p_right_sh.x * w), int(p_right_sh.y * h)
        mx, my = int((x_l + x_r) / 2), int((y_l + y_r) / 2)
        if overlay is not None:
            try:
                cv2.line(overlay, (x_l, y_l), (x_r, y_r), (255,0,0), 3)
                cv2.line(overlay, (x_n, y_n), (mx, my), (0,255,255), 3)
            except Exception: pass
        dY = y_l - y_r
        dX = x_l - x_r if (x_l - x_r) != 0 else 1e-6
        r_tilt = math.degrees(math.atan2(dY, dX))
        if face_landmarks:
            chin_y = face_landmarks[152].y * h
            head_top_y = face_landmarks[10].y * h
            face_h = abs(chin_y - head_top_y)
            if face_h < 1: face_h = 1
            d_slump = (my - y_n) / face_h
        else:
            d_slump = DUMMY_VALUE
        return d_slump, r_tilt
    except Exception:
        return DUMMY_VALUE, DUMMY_VALUE

In [10]:
# Process single image and return features in required order; also save annotated preview when requested
from IPython.display import display, Image as IPyImage

def process_and_annotate(image_path, save_preview=True, preview_dir=PREVIEW_DIR):
    img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError(f'Cannot read image: {image_path}')
    h, w = img.shape[:2]
    overlay = img.copy()
    mp_face_mesh = mp.solutions.face_mesh
    mp_pose = mp.solutions.pose
    mp_drawing = mp.solutions.drawing_utils
    mp_styles = mp.solutions.drawing_styles
    with mp_face_mesh.FaceMesh(refine_landmarks=True, max_num_faces=1) as fm, mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pm:
        rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        fres = fm.process(rgb)
        pres = pm.process(rgb)
        face_detected = bool(getattr(fres, 'multi_face_landmarks', None))
        pose_detected = bool(getattr(pres, 'pose_landmarks', None))
        face_lms = fres.multi_face_landmarks[0] if face_detected else None
        pose_lms = pres.pose_landmarks if pose_detected else None
        # draw face mesh if present
        if face_detected:
            try:
                mp_drawing.draw_landmarks(image=overlay, landmark_list=face_lms, connections=mp_face_mesh.FACEMESH_TESSELATION, landmark_drawing_spec=None, connection_drawing_spec=mp_styles.get_default_face_mesh_tesselation_style())
            except Exception: pass
        # compute numeric features
        ear = mar = pitch = yaw = roll = d_slump = r_tilt = DUMMY_VALUE
        eye_closure = DUMMY_VALUE
        if face_detected and face_lms is not None:
            fl = face_lms.landmark
            pts = [(lm.x * w, lm.y * h) for lm in fl]
            try:
                left_eye = [pts[i] for i in LEFT_EYE_INDICES]
                right_eye = [pts[i] for i in RIGHT_EYE_INDICES]
                ear = (eye_aspect_ratio(left_eye) + eye_aspect_ratio(right_eye)) / 2.0
            except Exception: ear = DUMMY_VALUE
            try:
                mouth_pts = [pts[i] for i in MOUTH_INDICES]
                mar = mouth_aspect_ratio(mouth_pts)
            except Exception: mar = DUMMY_VALUE
            try:
                # eye closure: vertical distance normalized by face width
                eye_v = (dist.euclidean(left_eye[1], left_eye[5]) + dist.euclidean(right_eye[1], right_eye[5])) / 2.0
                xs = [p[0] for p in pts]
                face_w = (max(xs) - min(xs)) if xs else 1.0
                eye_closure = eye_v / (face_w + 1e-6)
            except Exception: eye_closure = DUMMY_VALUE
            try: pitch, yaw, roll = calculate_geometric_head_pose(fl, w, h, overlay)
            except Exception: pitch = yaw = roll = DUMMY_VALUE
        if pose_detected and pose_lms is not None:
            try:
                pl = pose_lms.landmark
                d_slump, r_tilt = calculate_slump_geometry(pl, fl if face_detected else None, w, h, overlay)
            except Exception: d_slump = r_tilt = DUMMY_VALUE
        # annotate summary text on overlay with all features, using smaller font and tighter spacing
        texts = [
            f'Face:{int(face_detected)} Pose:{int(pose_detected)}',
            f'EAR:{round(ear,4) if ear!=DUMMY_VALUE else "D"} MAR:{round(mar,4) if mar!=DUMMY_VALUE else "D"}',
            f'PITCH:{round(pitch,4) if pitch!=DUMMY_VALUE else "D"} YAW:{round(yaw,4) if yaw!=DUMMY_VALUE else "D"}',
            f'ROLL:{round(roll,4) if roll!=DUMMY_VALUE else "D"} D_SLUMP:{round(d_slump,4) if d_slump!=DUMMY_VALUE else "D"}',
            f'R_TILT:{round(r_tilt,4) if r_tilt!=DUMMY_VALUE else "D"} EYE_CL:{round(eye_closure,4) if eye_closure!=DUMMY_VALUE else "D"}'
        ]
        for i, t in enumerate(texts):
            cv2.putText(overlay, t, (10, 30 + i*18), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1)
        # save preview if requested
        if save_preview:
            outname = os.path.basename(image_path)
            outpath = os.path.join(preview_dir, outname)
            try: cv2.imwrite(outpath, overlay)
            except Exception as e: print('Failed to write preview', outpath, e)
        # return ordered feature list
        features = [float(round(ear,4)) if ear!=DUMMY_VALUE else DUMMY_VALUE,
                    float(round(mar,4)) if mar!=DUMMY_VALUE else DUMMY_VALUE,
                    float(round(pitch,4)) if pitch!=DUMMY_VALUE else DUMMY_VALUE,
                    float(round(yaw,4)) if yaw!=DUMMY_VALUE else DUMMY_VALUE,
                    float(round(roll,4)) if roll!=DUMMY_VALUE else DUMMY_VALUE,
                    float(round(d_slump,4)) if d_slump!=DUMMY_VALUE else DUMMY_VALUE,
                    float(round(r_tilt,4)) if r_tilt!=DUMMY_VALUE else DUMMY_VALUE,
                    float(round(eye_closure,4)) if eye_closure!=DUMMY_VALUE else DUMMY_VALUE,
                    int(face_detected), int(pose_detected)]
        return features, overlay

In [11]:
# Batch processor: iterate dataset folders, process images, save up to 10 previews per class, export CSV
desired_map = {'Awake': 0, 'Sleep': 1, 'Yawning': 2, 'Passed_out': 3}
rows = []
preview_counts = {v:0 for v in desired_map.values()}
# Walk dataset folders
for folder in sorted(os.listdir(DATA_ROOT)):
    folder_path = os.path.join(DATA_ROOT, folder)
    if not os.path.isdir(folder_path): continue
    label = desired_map.get(folder, None)
    if label is None:
        print('Skipping unknown folder (no mapping):', folder)
        continue
    imgs = [f for f in sorted(os.listdir(folder_path)) if f.lower().endswith(('.jpg','.jpeg','.png','.bmp'))]
    print(f'Processing {folder} (label={label}) - {len(imgs)} images')
    saved_for_class = 0
    for fname in imgs:
        path = os.path.join(folder_path, fname)
        try:
            save_preview = (preview_counts[label] < 10)
            features, overlay = process_and_annotate(path, save_preview=save_preview, preview_dir=PREVIEW_DIR)
            if save_preview:
                # rename preview to include label and class count
                dst = os.path.join(PREVIEW_DIR, f'{folder}__{preview_counts[label]}__{fname}')
                try: os.replace(os.path.join(PREVIEW_DIR, os.path.basename(path)), dst)
                except Exception:
                    # if original name was different (e.g., we saved overlay as same name) attempt to write explicit file instead
                    try: cv2.imwrite(dst, overlay)
                    except Exception: pass
                preview_counts[label] += 1
            row = {
                'EAR': features[0], 'MAR': features[1], 'PITCH': features[2], 'YAW': features[3], 'ROLL': features[4],
                'D_SLUMP': features[5], 'R_TILT': features[6], 'EYE_CL': features[7], 'FACIAL_DISPLAYED': features[8], 'POSE_DISPLAYED': features[9],
                'LABEL': label, 'Image_Path': path
            }
            rows.append(row)
        except Exception as e:
            print('Skipped', path, '->', e)

# Export CSV
if rows:
    df = pd.DataFrame(rows)
    # round numeric columns where appropriate
    for c in ['EAR','MAR','PITCH','YAW','ROLL','D_SLUMP','R_TILT','EYE_CL']:
        if c in df.columns:
            df[c] = df[c].astype(float).round(4)
    out_csv = os.path.join(OUTPUT_DIR, 'training-feature.csv')
    df.to_csv(out_csv, index=False)
    print('Wrote', out_csv, 'rows=', len(df))
else:
    print('No rows to export')

Processing Awake (label=0) - 410 images
Processing Passed_out (label=3) - 237 images
Processing Sleep (label=1) - 530 images
Processing Yawning (label=2) - 892 images
Wrote c:\Code\Year_2\AI_Drowsiness_Detection\Outputs\training-feature.csv rows= 2069


In [13]:
df = pd.read_csv("./Outputs/training-feature.csv", header=0)

In [29]:
df[df['FACIAL_DISPLAYED'] == 0]

,EAR,MAR,PITCH,YAW,ROLL,D_SLUMP,R_TILT,EYE_CL,FACIAL_DISPLAYED,POSE_DISPLAYED,LABEL,Image_Path
410,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0000,-999.0,0,1,3,c:\Code\Year_2\AI_Drowsiness_Detection\Dataset...
412,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-1.2103,-999.0,0,1,3,c:\Code\Year_2\AI_Drowsiness_Detection\Dataset...
413,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,4.6355,-999.0,0,1,3,c:\Code\Year_2\AI_Drowsiness_Detection\Dataset...
414,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1.8596,-999.0,0,1,3,c:\Code\Year_2\AI_Drowsiness_Detection\Dataset...
417,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,5.5632,-999.0,0,1,3,c:\Code\Year_2\AI_Drowsiness_Detection\Dataset...
...,...,...,...,...,...,...,...,...,...,...,...,...
1490,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-5.1944,-999.0,0,1,2,c:\Code\Year_2\AI_Drowsiness_Detection\Dataset...
1491,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-7.8079,-999.0,0,1,2,c:\Code\Year_2\AI_Drowsiness_Detection\Dataset...
1492,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-7.8079,-999.0,0,1,2,c:\Code\Year_2\AI_Drowsiness_Detection\Dataset...
1809,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0000,-999.0,0,1,2,c:\Code\Year_2\AI_Drowsiness_Detection\Dataset...


In [34]:
# Filter dataframe:
# 1) Drop any images with LABEL==2 (Yawning) when FACIAL_DISPLAYED == 0
# 2) Change LABEL from 1 (Sleep) to 3 (Passed_out) when FACIAL_DISPLAYED == 0
df_filtered = df.copy()
# Drop rows where label==2 and no face displayed
df_filtered = df_filtered[~((df_filtered['LABEL'] == 2) & (df_filtered['FACIAL_DISPLAYED'] == 0))]
# Relabel rows where label==1 and no face displayed -> 3
mask = (df_filtered['LABEL'] == 1) & (df_filtered['FACIAL_DISPLAYED'] == 0)
if mask.any():
    df_filtered.loc[mask, 'LABEL'] = 3
# Summary counts after filtering
print('Counts by LABEL after filtering:', df_filtered['LABEL'].value_counts())

Counts by LABEL after filtering: LABEL
2    886
1    529
0    410
3    238
Name: count, dtype: int64


In [49]:
df_filtered[(df_filtered["LABEL"] == 0) & (df_filtered["FACIAL_DISPLAYED"] == 1)].count()

EAR                 410
MAR                 410
PITCH               410
YAW                 410
ROLL                410
D_SLUMP             410
R_TILT              410
EYE_CL              410
FACIAL_DISPLAYED    410
POSE_DISPLAYED      410
LABEL               410
Image_Path          410
dtype: int64

In [ ]:
# Finalize numeric columns and export CSVs
df_filtered.iloc[:, 0:8] = df_filtered.iloc[:, 0:8].round(4)
# keep the original training-feature export (OUTPUT_CSV) and also write the final.csv as requested
df_filtered.to_csv(OUTPUT_CSV, index=False)
OUTPUT_FINAL = os.path.join(OUTPUT_DIR, 'final.csv')
df_filtered.to_csv(OUTPUT_FINAL, index=False)
print('Wrote', OUTPUT_CSV, 'and', OUTPUT_FINAL, 'rows:', len(df_filtered))
display(df_filtered.head())

Wrote c:\Code\Year_2\AI_Drowsiness_Detection\Outputs\training-feature.csv rows: 2063


,EAR,MAR,PITCH,YAW,ROLL,D_SLUMP,R_TILT,EYE_CL,FACIAL_DISPLAYED,POSE_DISPLAYED,LABEL,Image_Path
0,0.3553,0.6065,36.7948,-2.4032,-5.2811,1.0014,0.5907,0.0651,1,1,0,c:\Code\Year_2\AI_Drowsiness_Detection\Dataset...
1,0.2809,0.2453,25.4717,-5.7497,-1.4201,0.9694,-4.8035,0.0543,1,1,0,c:\Code\Year_2\AI_Drowsiness_Detection\Dataset...
2,0.4133,0.2699,133.0847,-1.0480,-1.7680,0.7708,-1.2364,0.0862,1,1,0,c:\Code\Year_2\AI_Drowsiness_Detection\Dataset...
3,0.3716,0.2621,45.8079,-21.1239,4.3241,1.0824,4.0856,0.0693,1,1,0,c:\Code\Year_2\AI_Drowsiness_Detection\Dataset...
4,0.3518,0.4455,51.9544,-1.9713,0.0185,0.9944,0.5968,0.0658,1,1,0,c:\Code\Year_2\AI_Drowsiness_Detection\Dataset...
